In [1]:
# Imports
import pandas as pd
import numpy as np
import sklearn
from sklearn import pipeline, ensemble
import os

In [2]:
# File Reading and Splitting

# Read files
data = []
labels = []
longest_sample_length = 0

for root, dirpath, files in os.walk("Spectrograms"):
        for folder in dirpath:
            for r, d, files_in_folder in os.walk(os.path.join("Spectrograms", folder)):
                for f in files_in_folder:
                    path = os.path.join(root, folder, f)
                    d = np.load(path)
                    
                    
                    if len(d[1]) > longest_sample_length:
                        longest_sample_length = len(d[1])
                    
                    data.append(d)
                    labels.append(folder)
                    

data_new = []
for i in range(len(data)):
    d = data[i][1]
    num_samp = longest_sample_length // len(d)
    remainder = longest_sample_length % len(d)
    
    temp = []
    for j in range(num_samp):
        temp.extend(d)
    temp.extend(d[:remainder])
    
    data_new.append(temp)

print("Finished converting and padding data")

dx = [data_new, labels]
dx = np.swapaxes(dx, 0, 1)

split2 = 10 / 90
train_val, test = sklearn.model_selection.train_test_split(dx, test_size=0.10, random_state=42)
train, val = sklearn.model_selection.train_test_split(train_val, test_size=split2)

print("Split Complete! Train Size:", len(train), "Validation Size:", len(val), "Test Size:", len(test))

Finished converting and padding data
Split Complete! Train Size: 536 Validation Size: 67 Test Size: 

/opt/conda/envs/common/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


67


In [3]:
# Rotate all on axes
train = np.swapaxes(train, 0, 1)
val = np.swapaxes(val, 0, 1)

In [16]:
tdf = pd.DataFrame()
vdf = pd.DataFrame()

for i in range(len(train[0])):
    tdf[i] = train[0][i]
for i in range(len(val[0])):
    vdf[i] = val[0][i]
    
tdf = np.swapaxes(tdf, 0, 1)
vdf = np.swapaxes(vdf, 0, 1)

tdf.head()

,0,1,2,3,4,5,6,7,8,9,...,86123,86124,86125,86126,86127,86128,86129,86130,86131,86132
0,47.494743,12.753750,-41.730682,-47.414604,-45.326073,-47.394066,-39.798836,-38.789417,-46.857689,-41.487434,...,-49.004578,-50.709785,-59.730110,-53.875160,-31.037186,16.710194,42.422569,47.494743,12.753750,-41.730682
1,0.000000,0.000000,0.782850,-0.165725,-7.885173,-57.805275,-69.440453,-65.621368,-42.849693,-23.529236,...,-62.231281,-62.709221,-65.466644,-63.971794,-48.395561,-27.523640,-17.027130,-11.119843,-7.478520,-3.465441
2,40.904423,41.818329,36.357288,28.173731,28.495445,32.168991,35.340904,28.641685,30.270784,28.381844,...,-16.789532,-20.700569,-14.813272,-13.408361,-13.614334,-8.178479,-10.265069,-5.017176,0.352041,-11.677095
3,47.417320,51.310577,48.132950,46.865685,48.761986,44.517586,43.386887,37.461491,37.787323,48.121017,...,-60.411369,-65.200806,-67.444626,-71.243683,-73.427826,-73.664658,-75.748222,-80.852806,-80.326904,-79.200546
4,-37.839466,-31.705290,-23.682901,-19.411900,-17.922112,-15.011089,-19.800613,-11.344162,-10.192379,-20.317223,...,-4.763318,-23.394947,-53.059898,-56.254509,-51.947594,-25.978527,-11.424651,-16.848080,-8.525493,-7.364831


In [20]:
pipeline = sklearn.pipeline.make_pipeline(sklearn.ensemble.RandomForestClassifier())

X = tdf
y = train[1]

pipeline.fit(X, y)

valpred = pipeline.predict(vdf)

v_acc = sklearn.metrics.accuracy_score(valpred, val[1])

print("Validation:", v_acc)

Validation: 0.4626865671641791
